In [4]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

from pyshadow.main import Shadow

# FUNCIONES SCRAPING:

- URLS

In [2]:
def iniciar_scraper(user_agent, urls):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    sleep(5)

    urls_paginas_perfumes = []
    
    inicio = 1
    
    for url in urls:
        
        scrape_url = url
        browser.get(scrape_url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.CLASS_NAME, "amgdprcookie-modal__link").click()
            sleep(6)########################################################################

        urls_paginas_perfumes.append(scrape_url)

        while True:

            if not browser.current_url == scrape_url:
                browser.get(scrape_url)
                sleep(5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            try:
                scrape_url = soup.find("ul", class_= "pager__list").find_all("li")[-1].find("a")["href"]
                urls_paginas_perfumes.append(scrape_url)
                sleep(1)
            except:
                break
                
        inicio += 1
    ############################################################################################################    
    browser.close()    
    return urls_paginas_perfumes

- FUNCIÓN INFO:

In [19]:
def scrape_info(user_agent, url):
    
    response = requests.get(url)
    sleep(10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    productos = soup.find_all("div", class_= "product-item-info")
    
    cat = soup.find("h2", class_= "category-info__title").text
    
    info_productos = []
    
    for producto in productos:
        
        categoria = cat
        
        try:
            marca = producto.find("div", class_= "product-grid-item__brand").text
        except:
            marca = "Sin marca"
            
        try:
            nombre = producto.find("div", class_= "product-grid-item__name").text.strip()
        except:
            nombre = "Sin nombre"
            
        try:
            tipo = producto.find("div", class_= "product description product-item-description").text.strip()
        except:
            tipo = "Sin categoria"       
                 
        try:
            precio_sin_desc = producto.find("span", class_= "price__value").text.strip().replace("\xa0", "").replace("Precio habitual\n\n", "").strip()
        except:
            precio_sin_desc = "Sin precio"
            
        try:
            descuento = producto.find("span", class_= "price__value--percent").text.strip().replace("(", "").replace(")", "")
        except:
            descuento = "Sin descuento"
            
        try:
            precio_con_desc = producto.find("span", class_= "price").text.strip().replace("\xa0", "")
        except:
            precio_con_desc = "Sin precio descuento"
        
        
        try:
            url_producto = producto["data-link"]
            #################################################
            # Options:
            opts = Options()
            opts.add_argument("--disable-extensions")
            opts.add_argument(f"User-Agent={user_agent}")
            # Initialize:
            browser = webdriver.Chrome(options=opts)
            sleep(3)
            browser.get(url_producto)
            sleep(5)            
            browser.find_element(By.CLASS_NAME, "amgdprcookie-modal__link").click()
            sleep(3)
            shadow = Shadow(browser)
            
            valoracion = shadow.find_elements('span[class="grade"]')[0].text
            num_valoraciones = shadow.find_elements('span[class="count"]')[0].text.replace("(", "").replace(")", "")
            browser.close()
            #################################################
            
        except:
            valoracion = "Sin valoracion"
            num_valoraciones = "Sin valoraciones"
        
        info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])

    return info_productos   

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

# Scrapeo URLS

In [3]:
# PONER EL PATH Y EL USER-AGENT PROPIOS:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

urls = ["https://www.primor.eu/es_es/perfumes-para-hombres", "https://www.primor.eu/es_es/perfumes-de-mujer"]

In [4]:
urls_paginas_perfumes = iniciar_scraper(user_agent, urls)

['https://www.primor.eu/es_es/perfumes-para-hombres',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=2',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=3',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=4',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=5',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=6',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=7',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=8',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=9',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=10',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=11',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=12',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=13',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=14',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=15',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=16',
 'https://www.primor.eu/es_es/perfumes-para-hombres?p=17',
 'https://

In [6]:
df_urls_paginas_perfumes = pd.DataFrame(urls_paginas_perfumes, columns= ["url"])

## CSV

In [8]:
df_urls_paginas_perfumes.to_csv("urls_paginas_primor.csv", index= False, sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

# Scrapeo la info de todos los productos de cada página:

In [9]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

In [ ]:
# urls = pd.read_csv("urls_paginas_primor.csv")

In [25]:
info_productos = []

for url in tqdm.tqdm(urls[:25]):

    info = scrape_info(user_agent, url)
    info_productos.extend(info)

  0%|          | 0/25 [00:00<?, ?it/s]C:\Users\regue\AppData\Local\Temp\ipykernel_16532\3148243916.py:56: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path = path_driver, chrome_options=opts)
100%|██████████| 25/25 [2:54:22<00:00, 418.50s/it]  


In [26]:
df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"] )

In [28]:
df_info_productos

,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,Perfumes para hombres,Hugo Boss,Boss Bottled EDT,Sin categoria,"81,70€",-56%,"35,95€","4,81",212
1,Perfumes para hombres,Davidoff,Cool Water Hombre,Sin categoria,"74,00€",-74%,"18,94€","4,71",118
2,Perfumes para hombres,Armani,Giorgio Armani Perfume Hombre Acqua Di Gio Hom...,Sin categoria,"85,00€",-54%,"38,94€","4,93",98
3,Perfumes para hombres,Calvin Klein,Ck In2u For Him EDT,Sin categoria,"76,00€",-76%,"18,49€","4,70",94
4,Perfumes para hombres,Carolina Herrera,212 Men EDT,Sin categoria,"74,50€",-49%,"37,95€","4,89",66
...,...,...,...,...,...,...,...,...,...
595,Perfumes para hombres,Cartier,L'Envol EDP,Sin categoria,"85,00€",Sin descuento,"63,99€","5,00",1
596,Perfumes para hombres,Cartier,Déclaration Haute Fraîcheur Eau de Toilette,Sin categoria,"82,00€",Sin descuento,"58,99€","4,00",2
597,Perfumes para hombres,4711,Acqua Colonia Lychee & White Mint,Sin categoria,"57,00€",-50%,"28,64€","4,00",1
598,Perfumes para hombres,Dior,EAU SAUVAGE EXTRÊME Eau de Toilette Intense,Sin categoria,"136,00€",Sin descuento,"101,95€",5,1


## CSV:

In [27]:
df_info_productos.to_csv("datos_perfumes_primor.csv", index="False", sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################